# Deep learning models

### Todo



#### This cell is for defining various OPTIONS used for this notebook (working directory, how many rows and columns pandas displays for a dataframe, etc). 

#### Preferably this cell is also where we do important imports (for example pandas and numpy)

In [8]:
import os 
#Input the directory where your joined_data.csv is located 
#os.chdir('C:/Users/Trond/Documents/Master 2020/Processed data')
os.chdir('C:/Users/Briggstone/Documents/Master 2020/Processed data')
#os.chdir('C:/Users/MyPC/Documents/Andrijana/UiS/DATMAS Master oppgave/Processed data')

#Where you want the csv file of the merged data to be placed
output_filepath = 'C:/Users/Briggstone/Documents/Master 2020/Processed data'
#output_filepath = 'C:/Users/MyPC/Documents/Andrijana/UiS/DATMAS Master oppgave/Processed data'

import numpy as np
import pandas as pd

# Set ipython's max row display
pd.set_option('display.max_row', 1000)

# Set iPython's max column width to 50
pd.set_option('display.max_columns', 50)

# Which folder of preprocessed data we want to use
FOLDER_NAME = "Deep_Learning_Approach"


#### In this cell we define additional preprocessing functions specific for LSTM

In [9]:
from sklearn.model_selection import train_test_split

def create_generator_arrays(df):
    
    df_x = []
    df_y = []
    
    unique_subjects = df.PATNO.unique()  
    
    for u in unique_subjects:
        
        subject_data = df.loc[df.PATNO == u, :]
        
        y = subject_data.tail(1).HALL.values[0]
        y = np.expand_dims(y, axis = 0)
        df_y.append(y)
        
        subject_data.drop(["HALL", "PATNO"], axis = 1, inplace = True)
        
        x = subject_data.to_numpy()
        x = np.expand_dims(x, axis=0)
        df_x.append(x)
        
    return df_x,df_y



#Need to generate a validation set as we use this for early stopping and model validation for deep learning
#Generates a validation set of equal size to test set
def train_validation_test_split (train,test):
    
    num_of_unique_subjects_train = train.PATNO.unique().size  
    num_of_unique_subjects_test = test.PATNO.unique().size  
    total_unique_subjects = num_of_unique_subjects_train + num_of_unique_subjects_test
    
    train_ratio = num_of_unique_subjects_train / total_unique_subjects
    test_ratio = num_of_unique_subjects_test / total_unique_subjects
    
    #ratio to take from training set to make validation as large as test set
    
    validation_ratio = test_ratio / train_ratio
    
    tempdata = pd.DataFrame(train.PATNO.unique(), columns = ["PATNO"])

    HALL_EVER = []
    for id in train.PATNO.unique():

        if train.loc[(train.PATNO == id) & (train.HALL == 1), "HALL"].empty:
            HALL_EVER.append(0)
        else:
            HALL_EVER.append(1)

    Y = HALL_EVER
    X = tempdata

    temptrain, validation, _, _ = train_test_split( X, Y, test_size= validation_ratio, random_state= 1, stratify= Y, shuffle = True)

    temptrain = train.merge(temptrain, how = "inner", on = "PATNO")
    validation = train.merge(validation, how = "inner", on = "PATNO")
    
    train = temptrain
    
    return train,validation,test
    

#### In this cell we do additional preprocessing specific for LSTM

In [10]:
train = pd.read_csv(FOLDER_NAME + '/train.csv')
test = pd.read_csv(FOLDER_NAME + '/test.csv')

# We don't implement samling for deep learning due to poor results on other algorithms
train.drop(["SAMPLING"], axis = 1, inplace  = True)

train,validation,test = train_validation_test_split (train,test)

train_x, train_y = create_generator_arrays(train)
validation_x, validation_y = create_generator_arrays(validation)
test_x, test_y = create_generator_arrays(test)


num_of_dims = train.shape[1] - 2
num_of_unique_subjects_train = train.PATNO.unique().size  
num_of_unique_subjects_validation = validation.PATNO.unique().size
num_of_unique_subjects_test = test.PATNO.unique().size  

print("Number of subjects train: ", num_of_unique_subjects_train)
print("Number of subjects validation: ", num_of_unique_subjects_validation)
print("Number of subjects test: ", num_of_unique_subjects_test)

C:\Users\Briggstone\AppData\Roaming\Python\Python37\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Number of subjects train:  277
Number of subjects validation:  61
Number of subjects test:  61


#### In this section we define helper functions for the LSTM implementation

In [176]:
def data_generator (x,y, num_of_epochs):
    for e in range(num_of_epochs):
        for i in range (len(x)):
            yield x[i], y[i]
            
def scorer_helper_function(y_true,y_pred,scorer_string):
    
    if scorer_string == "accuracy":
        return met.accuracy_score(y_true, y_pred)
    elif scorer_string == "precision":
        return met.precision_score(y_true,y_pred)
    elif scorer_string == "sensitivity":
        return met.recall_score(y_true,y_pred)
    elif scorer_string == "specificity":
        return met.recall_score(y_true,y_pred, pos_label = 0)
    elif scorer_string == "f1":
        return met.f1_score(y_true,y_pred)
    elif scorer_string == "roc_auc":
        return met.roc_auc_score(y_true,y_pred)
    elif scorer_string == "MCC":
        return met.matthews_corrcoef(y_true,y_pred)
        
        
def search_report (predictions, true_y, print_all = False, refit = 'sensitivity'):
    
        METRICS = {
        'accuracy': met.make_scorer(met.accuracy_score),
        'precision': met.make_scorer(met.precision_score),
        'sensitivity': met.make_scorer(met.recall_score),
        'specificity': met.make_scorer(met.recall_score,pos_label = 0),
        'f1': met.make_scorer(met.f1_score),
        'roc_auc': met.make_scorer(met.roc_auc_score),
        "MCC" : met.make_scorer(met.matthews_corrcoef)
    }
        
    columns = ["Result_Type"]
    for k,_ in FINAL_RESULTS_METRICS.items():
        columns.append(k)
    
    
    df = pd.DataFrame(columns = columns)
    rows_list = []
    
    for predicition in predictions:
    
    

#### In this cell we implement a simple LSTM with one layer and test different layer sizes

In [178]:
from keras.models import Sequential 
from keras.layers import LSTM, Dense, Dropout

from keras.callbacks.callbacks import EarlyStopping, ModelCheckpoint
import keras.metrics

import keras.optimizers as optimizers

layer_sizes = [8,12,16,20,24,28,32,36]

validation_predictions_layers = []

for layer_size in layer_sizes:

    model = Sequential (name = "LSTM")

    model.add(LSTM(layer_size, input_shape=(None, num_of_dims)))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy',
                  metrics=['accuracy',keras.metrics.Recall(name = "sensitivity")])


    from sklearn.utils import class_weight

    train_y_temp = [y.item() for y in train_y]

    class_weights = class_weight.compute_class_weight('balanced',
                                                     np.unique(train_y_temp),
                                                     train_y_temp)


    es = EarlyStopping(monitor='val_sensitivity', mode = 'max', verbose = 1, patience = 100)
    mc = ModelCheckpoint('best_model.h5', monitor = 'val_sensitivity', mode = 'max', save_best_only = True, verbose = 1, save_weights_only= True)

    cb_list = [es,mc]

    epochs = 500

    model.fit_generator(data_generator(train_x,train_y, epochs), steps_per_epoch = num_of_unique_subjects_train, epochs = epochs, verbose = 0, class_weight=class_weights, \
                       validation_data = data_generator(validation_x,validation_y, epochs), validation_steps= num_of_unique_subjects_validation, callbacks = cb_list)
    
    model.load_weights('best_model.h5')

    predictions = model.predict_generator(data_generator(validation_x, validation_y, 1), steps = num_of_unique_subjects_validation) > 0.5
    
    validation_predictions_layers.append(predictions)

Model: "LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_33 (LSTM)               (None, None, 12)          2256      
_________________________________________________________________
dropout_27 (Dropout)         (None, None, 12)          0         
_________________________________________________________________
lstm_34 (LSTM)               (None, None, 12)          1200      
_________________________________________________________________
dropout_28 (Dropout)         (None, None, 12)          0         
_________________________________________________________________
lstm_35 (LSTM)               (None, 12)                1200      
_________________________________________________________________
dropout_29 (Dropout)         (None, 12)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 13     

In [179]:
model.load_weights('best_model.h5')

predictions = model.predict_generator(data_generator(test_x, test_y, 1), steps = num_of_unique_subjects_test) > 0.5

In [180]:
from sklearn.metrics import confusion_matrix

confusion_matrix(validation_y, predictions)

array([[32, 14],
       [ 9,  6]], dtype=int64)

In [181]:
from sklearn import metrics as met

print(met.accuracy_score(test_y, predictions))
print(met.recall_score(test_y, predictions))
print(met.f1_score(test_y, predictions))
print(met.roc_auc_score(test_y, predictions))

0.6229508196721312
0.4
0.34285714285714286
0.5478260869565217
